In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

from tqdm import tqdm
from pathlib import Path 

In [18]:
path = Path.cwd()
file = pd.read_excel(path/'keyword.xlsx',engine='openpyxl')
category_list = file.values.tolist()

In [21]:
column = ['업무','공고번호-차수','분류','공고명','공고기관','수요기관','계약방법','입력일시','입찰마감일시']
df = pd.DataFrame(columns=column)
keywords = []
try:
    driver = webdriver.Chrome()
    driver.implicitly_wait(2)

    for query in category_list:

        
        
        driver.get('https://www.g2b.go.kr:8101/ep/tbid/tbidFwd.do')
        task_dict = {'용역': 'taskClCds5'}
        for task in task_dict.values():
            checkbox = driver.find_element(By.ID, task)
            checkbox.click()
        # ID값이 bidNm인 태그 가져오기
        bidNm = driver.find_element(By.ID, 'bidNm')
        # 내용을 삭제
        bidNm.clear()
        # 검색어에 문자열 전달
        bidNm.send_keys(query)
        bidNm.send_keys(Keys.RETURN)
        
        option_dict = {'검색기간 1달': 'setMonth1_1', '입찰마감건 제외': 'exceptEnd', '검색건수 표시': 'useTotalCount'}
        for option in option_dict.values():
            checkbox = driver.find_element(By.ID, option)
            checkbox.click()

        # 목록수 100건 선택 (드롭다운)
        print(1)
        recordcountperpage = driver.find_element(By.NAME,'recordCountPerPage')
        selector = Select(recordcountperpage)
        selector.select_by_value('100')

        search_button = driver.find_element(By.CLASS_NAME, 'btn_mdl')
        search_button.click()
        print(2)
        # 검색 결과 확인
        elem = driver.find_element(By.CLASS_NAME,'results')
        div_list = elem.find_elements(By.TAG_NAME, 'tr')

        # 리스트 형태로 검색결과 저장
        
        print(3)
        results = []
        links = []
        for div in tqdm(div_list):
            results.append(div.text)
            row_data = [div.text.split('\n')]
            a_tags = div.find_elements(By.TAG_NAME,'a')
            if a_tags:
                a_tag = a_tags[0]
                link = a_tag.get_attribute('href')
                row_data.append([link])
                keywords.append(query)
            # create a new DataFrame with the data from this row
            
            df_row = pd.DataFrame(row_data)
            # append it to the overall df using pd.concat()
            df = pd.concat([df, df_row], ignore_index=True)
                #df = df.assign(링크 =links)

except Exception as e:
    print(e)
finally:  
    driver.quit()

1
2
3


100%|██████████| 68/68 [00:19<00:00,  3.42it/s]


1
2
3


100%|██████████| 6/6 [00:02<00:00,  2.30it/s]


1
2
3


100%|██████████| 11/11 [00:03<00:00,  3.55it/s]


1
2
3


100%|██████████| 5/5 [00:02<00:00,  2.15it/s]


1
2
3


100%|██████████| 2/2 [00:04<00:00,  2.10s/it]


In [22]:
df = df[~df[0].isin(['업무 공고번호-차수 분류 공고명 공고기관 수요기관 계약방법 입력일시','검색된 데이터가 없습니다.'])]
df

,업무,공고번호-차수,분류,공고명,공고기관,수요기관,계약방법,입력일시,입찰마감일시,0,1,2,3,4,5,6,7,8,9
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용역,20240712711-00,일반,[스마트ICT] 5차년도 USG AI·데이터 문제해결 제조혁신 경진대회 운영 용역,경남대학교,경남대학교,제한(총액)협상에의한계약,2024/07/10 10:32,(2024/07/23 11:00),NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.g2b.go.kr:8081/ep/invitation/publi...,None,None,None,None,None,None,None,None,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용역,20240712612-00,일반,2024년 국내주식 의안분석 데이터베이스 제공기관 선정,국민연금공단,국민연금공단,제한(총액)협상에의한계약,2024/07/10 10:21,(2024/07/22 10:00),NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.g2b.go.kr:8081/ep/invitation/publi...,None,None,None,None,None,None,None,None,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용역,20240712598-00,긴급,하천퇴적물 지화학 데이터를 이용한 딥러닝 기반 리튬 매장지 예측모델 개발,조달청 대전지방조달청,한국지질자원연구원,수의(총액),2024/07/10 10:19,(2024/07/10 12:00),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.g2b.go.kr:8101/ep/tbid/serviceBidD...,None,None,None,None,None,None,None,None,NaN
172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용역,20240635390-00,긴급,차세대 자율적 내부통제(청백-e) 정보시스템 마스터플랜(ISMP) 수립,조달청 서울지방조달청,한국지역정보개발원,일반(총액)협상에의한계약,2024/07/02 15:48,(2024/07/16 10:00),NaN
173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.g2b.go.kr:8081/ep/invitation/publi...,None,None,None,None,None,None,None,None,NaN
174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용역,20240632059-00,일반,내부통제체계 고도화 컨설팅 사업자 선정,서민금융진흥원,서민금융진흥원,제한(총액)협상에의한계약,2024/06/24 13:58,(2024/07/11 16:00),NaN


In [23]:
column = ['업무','공고번호-차수','분류','공고명','공고기관','수요기관','계약방법','입력일시','입찰마감일시','링크']
new_df = df.iloc[:,9:19]
new_df.columns = column

In [24]:
flatten_keyword = [item for sublist in keywords for item in sublist]

In [25]:
aa = new_df.iloc[::2,:]
with_link = pd.DataFrame(new_df.iloc[1::2,:]['업무'])
with_link = list(with_link['업무'])
aa['링크'] = with_link
aa.insert(0,'키워드',flatten_keyword)

C:\Users\FS868CS\AppData\Local\Temp\ipykernel_48868\412808289.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['링크'] = with_link


In [26]:
aa['입력일시'] = pd.to_datetime(aa['입력일시'])
aa = aa.sort_values('입력일시',ascending=False)
aa = aa.drop_duplicates(['공고번호-차수'])
aa = aa.reset_index(drop=True) 

C:\Users\FS868CS\AppData\Local\Temp\ipykernel_48868\2056726429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['입력일시'] = pd.to_datetime(aa['입력일시'])


In [27]:
from datetime import datetime 
today = datetime.today().strftime('%Y-%m-%d')

In [31]:
aa.to_excel(f'EYC_RC_{today}.xlsx')